In [1]:
import os 
import onnx
#!pip install onnxruntime==1.8.1
import onnxruntime
from torch.autograd import Function
import torch
from torch import nn
import numpy as np

In [4]:
import os
os.environ['ONNXRUNTIME_DIR']='D:\桌面\包\onnxruntime-linux-x64-1.8\onnxruntime-linux-x64-1.8.1'
os.environ['LD_LIBRARY_PATH']='D:\桌面\包\onnxruntime-linux-x64-1.8\onnxruntime-linux-x64-1.8.1\lib'

In [ ]:
!wget https://github.com/microsoft/onnxruntime/releases/download/v1.8.1/onnxruntime-linux-x64-1.8.1.tgz
!tar -zxvf onnxruntime-linux-x64-1.8.1.tgz

In [4]:
!mkdir plugin
%cd plugin
!mkdir cpu
%cd ..
!ls

D:\myjupyter\深度学习\deployment\onnx\plugin
D:\myjupyter\深度学习\deployment\onnx


'ls' 不是内部或外部命令，也不是可运行的程序
或批处理文件。


In [5]:
%cd plugin/cpu
!ls

D:\myjupyter\深度学习\deployment\onnx\plugin\cpu


'ls' 不是内部或外部命令，也不是可运行的程序
或批处理文件。


In [6]:
head_file ="""
#include "onnxruntime_register.h"
#include "custom.h"
#include "ort_mmcv_utils.h"

const char *c_MMCVOpDomain = "mmcv";
CustomOp  c_CustomOp;

OrtStatus *ORT_API_CALL RegisterCustomOps(OrtSessionOptions *options,
                                          const OrtApiBase *api) {
  OrtCustomOpDomain *domain = nullptr;
  const OrtApi *ortApi = api->GetApi(ORT_API_VERSION);

  if (auto status = ortApi->CreateCustomOpDomain(c_MMCVOpDomain, &domain)) {
    return status;
  }


  if (auto status = ortApi->CustomOpDomain_Add(domain, &c_CustomOp)) {
    return status;
  }

  return ortApi->AddCustomOpDomain(options, domain);
}

"""
with open('onnxruntime_register.cpp','w') as f:
    f.write(head_file)
head_file ="""
#include "custom.h"
#include <assert.h>
#include <algorithm>
#include <iostream>
#include <vector>

#include "../ort_mmcv_utils.h"

void CustomKernel::Compute(OrtKernelContext *context) {
    const float value = float(value_);
    const OrtValue *inputs = ort_.KernelContext_GetInput(context, 0);
    const float * inputs_data = reinterpret_cast<const float*>(ort_.GetTensorData<const float*>(inputs));
    
    OrtTensorDimensions input_dims(ort_, inputs); 
    std::cout<<input_dims.data()<<std::endl;
    int nums=1;
    int size = int(input_dims.size());
    for (int i =0;i<size;i++)
        nums *=input_dims.data()[i]; 
    
    OrtValue* output = ort_.KernelContext_GetOutput(context, 0, input_dims.data(), input_dims.size());
    float* out = ort_.GetTensorMutableData<float>(output);
    for (int index =0;index<nums;index++)
        out[index]=inputs_data[index] + value;
}

"""
with open('custom.cpp','w') as f:
    f.write(head_file)

In [7]:
%cd ..
!ls

D:\myjupyter\深度学习\deployment\onnx\plugin


'ls' 不是内部或外部命令，也不是可运行的程序
或批处理文件。


In [8]:
head_file = """
#ifndef ORT_MMCV_UTILS_H
#define ORT_MMCV_UTILS_H
#include <onnxruntime_cxx_api.h>

#include <vector>

struct OrtTensorDimensions : std::vector<int64_t> {
  OrtTensorDimensions(Ort::CustomOpApi ort, const OrtValue* value) {
    OrtTensorTypeAndShapeInfo* info = ort.GetTensorTypeAndShape(value);
    std::vector<int64_t>::operator=(ort.GetTensorShape(info));
    ort.ReleaseTensorTypeAndShapeInfo(info);
  }
};
#endif  // ORT_MMCV_UTILS_H

"""
with open('ort_mmcv_utils.h','w') as f:
    f.write(head_file)
    
head_file = """
#ifndef ONNXRUNTIME_REGISTER_H
#define ONNXRUNTIME_REGISTER_H
#include <onnxruntime_c_api.h>

#ifdef __cplusplus
extern "C" {
#endif

OrtStatus *ORT_API_CALL RegisterCustomOps(OrtSessionOptions *options,
                                          const OrtApiBase *api);

#ifdef __cplusplus
}
#endif
#endif  // ONNXRUNTIME_REGISTER_H
"""
with open('onnxruntime_register.h','w') as f:
    f.write(head_file)
head_file = """
#ifndef ONNXRUNTIME_CUSTOM_H
#define ONNXRUNTIME_CUSTOM_H

#include <onnxruntime_cxx_api.h>


struct CustomKernel {
  CustomKernel(OrtApi api, const OrtKernelInfo *info) : api_(api), ort_(api_), info_(info)
  {
    value_ = ort_.KernelInfoGetAttribute<float>(info, "value_add");

  // create allocator
  // allocator_ = Ort::AllocatorWithDefaultOptions();
  };

  void Compute(OrtKernelContext *context);

 protected:
  OrtApi api_;
  Ort::CustomOpApi ort_;
  const OrtKernelInfo *info_;
  // Ort::AllocatorWithDefaultOptions allocator_;
private:
  float value_;

};

struct CustomOp : Ort::CustomOpBase<CustomOp, CustomKernel> {
  void *CreateKernel(OrtApi api, const OrtKernelInfo *info) const {
    return new CustomKernel(api, info);
  };

  const char *GetName() const { return "Custom"; };

  size_t GetInputTypeCount() const { return 1; };
  ONNXTensorElementDataType GetInputType(size_t /*index*/) const {
    return ONNX_TENSOR_ELEMENT_DATA_TYPE_FLOAT;
  };

  size_t GetOutputTypeCount() const { return 1; };
  ONNXTensorElementDataType GetOutputType(size_t index) const {
    return ONNX_TENSOR_ELEMENT_DATA_TYPE_FLOAT;
  }

  // force cpu
  const char *GetExecutionProviderType() const {
    return "CPUExecutionProvider";
  }
};

#endif

"""
with open('custom.h','w') as f:
    f.write(head_file)

In [9]:
%cd ..
!ls

D:\myjupyter\深度学习\deployment\onnx


'ls' 不是内部或外部命令，也不是可运行的程序
或批处理文件。


In [5]:
head_file = """
import glob
import os
import re
from pkg_resources import DistributionNotFound, get_distribution, parse_version
from setuptools import find_packages, setup

from torch.utils.cpp_extension import BuildExtension
EXT_TYPE = 'pytorch'
cmd_class = {'build_ext': BuildExtension}

def get_extensions():
    import torch
    extensions = []
    ext_name = 'mmcv._ext_ort'
    from torch.utils.cpp_extension import library_paths, include_paths
    import onnxruntime
    library_dirs = []
    libraries = []
    include_dirs = []
    
    ort_path = os.getenv('ONNXRUNTIME_DIR', '0')
    library_dirs += [os.path.join(ort_path, 'lib')]
    libraries.append('onnxruntime')
    kwargs = {}
    define_macros = []
    extra_compile_args = {'cxx': []}
 
    include_path = os.path.abspath('./plugin/')
    include_dirs.append(include_path)
    include_dirs.append(os.path.join(ort_path, 'include'))
    

    op_files = glob.glob('./plugin/cpu/*')

    include_dirs += include_paths(cuda=False)
    library_dirs += library_paths(cuda=False)

    kwargs['library_dirs'] = library_dirs
    kwargs['libraries'] = libraries

    from setuptools import Extension
    ext_ops = Extension(
        name=ext_name,
        sources=op_files,
        include_dirs=include_dirs,
        define_macros=define_macros,
        extra_compile_args=extra_compile_args,
        language='c++',
        library_dirs=library_dirs,
        libraries=libraries)
    extensions.append(ext_ops)
    return extensions
setup(
    name='mmcv',
    ext_modules=get_extensions(),
    cmdclass=cmd_class,)

"""
with open('setup.py','w') as f:
    f.write(head_file)

In [ ]:
!python setup.py install

In [ ]:
import torch
torch.ops.load_library('/kaggle/working/onnxruntime-linux-x64-1.8.1/lib/libonnxruntime.so')
torch.ops.load_library('/kaggle/working/onnxruntime-linux-x64-1.8.1/lib/libonnxruntime.so.1.8.1')

In [ ]:
class custom(torch.autograd.Function):
    
    @staticmethod
    def symbolic(g, inputs,value):
        return g.op('mmcv::Custom',inputs,value_add_f=value)
    @staticmethod
    def forward(ctx,inputs,value):
        outputs =inputs +value
        return outputs
testfunc = custom.apply

In [ ]:
_value =3.0
def func(inputs):
    return testfunc(inputs,_value)

In [ ]:
from torch import nn
class WrapFunction(nn.Module):

    def __init__(self, wrapped_function):
        super(WrapFunction, self).__init__()
        self.wrapped_function = wrapped_function

    def forward(self, *args, **kwargs):
        return self.wrapped_function(*args, **kwargs)

In [ ]:
ort_custom_op_path='build/lib.linux-x86_64-3.7/mmcv/_ext_ort.cpython-37m-x86_64-linux-gnu.so'

In [ ]:
session_options = rt.SessionOptions()
if ort_custom_op_path:
    session_options.register_custom_ops_library(ort_custom_op_path)

In [ ]:
wrapped_model = WrapFunction(func)
wrapped_model.cpu().eval()

In [ ]:
inputs = torch.randn(1,3,3)

In [ ]:
with torch.no_grad():
    torch.onnx.export(wrapped_model,(inputs,),'sample.onnx',input_names=['inputs'])

In [ ]:
model = onnx.load('sample.onnx')

In [ ]:
sess = rt.InferenceSession('sample.onnx', session_options)

In [ ]:
onnx_output = sess.run(None, {
    'inputs': inputs.detach().numpy(),
})